In [1]:
import os
import logging
import sys
import nest_asyncio
import json

In [2]:
cd ..

d:\Project\Graph-RAG


In [3]:
from dotenv import load_dotenv
from llama_index.llms.openai import OpenAI
from llama_index.core import (
    Settings,
    Document, get_response_synthesizer,
    KnowledgeGraphIndex, PropertyGraphIndex,
    StorageContext
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.graph_stores.neo4j import Neo4jGraphStore

from core.data.processing import process_jsonl_data
from core.graph.create import instantiate

d:\Administrator\anaconda3\envs\rag\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
DATA_PATH = "core/data/annotations.jsonl"

In [5]:
chunks = process_jsonl_data(DATA_PATH)
chunks = chunks[:10]
len(chunks)

Loaded 74669 documents


Parsing nodes: 100%|██████████| 74669/74669 [00:20<00:00, 3626.98it/s]


Finish splitting 77319 chunks


Processing chunks: 100%|██████████| 77319/77319 [00:08<00:00, 9553.05it/s] 


Processed 77319 chunks


10

In [6]:
instantiate(chunks)

INFO:neo4j.notifications:Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:Entity) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_1ed05907 FOR (e:Entity) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: '\n                CREATE CONSTRAINT IF NOT EXISTS FOR (n:Entity) REQUIRE n.id IS UNIQUE;\n                '
Configurations loaded with username: neo4j at url: neo4j+s://fbff1d57.databases.neo4j.io
Finish indexing 10 nodes
INFO:neo4j.notifications:Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT unique_chunk IF NOT EXISTS FOR (e:Chunk) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT unique_chunk FOR (e:Chunk) REQUIRE (e.id) IS UNIQUE` alre